# Merck Graphical User Interface (GUI)

<font color='red'>Disclaimer: This is a pilot of the real tool.</font>

Notebook for Merck users to interact with the Natural Language to SQL tool.

**Note**: Before you start running the notebook, please make sure that you are using `conda_pytorch_latest_p36` conda environment. If not, please change it by clicking on:
```
Kernel -> Change Kernel -> conda_pytorch_latest_p36
```

In [ ]:
%load_ext autoreload
%autoreload 2
%cd ./src/

## Start the GUI 

The GUI will appear after running the cell below. To do so you can select the cell and click `▶︎⎮ Run`.

## Overview

    1. Set the credentials.
    2. Introduce the natural language query in the main tab. Hit "detect".
    3. Review that the query appearing in the box below is correct and entities are mapped to the correct ICD10 codes.
    4. (optional) If any corrections are required use the "correct detection" or "correct code map" tabs as needed.
    5. Hit "Execute" to see the requested table.


## In-depth

### Credentials interface

Merck require users to have access to the data in Redshift. To ensure the security of Merck data users are required to provide their individual credentials to use the tool. The credentials are used to "Execute" the queries against the DB. 


### Main interface

The interface is composed of three tabs: `Main`, `Correct detection` and `Correct code map`. 

* The `main` tab is used to input the Natural Language Query and has two buttons: 

    1. `Detect` button runs key entities detection and maps key entities to the most likely option.
    2. `Execute` is intended to be runned after the user verifies that the detection and disambiguation are correct. It will run the query agains the Data Base and return the desired table. 
    
* The `Correct detection` is intended for the user to add terms in the sentence that hasen't been automatically detected.

    1. The `Write name` box is intended for the user to write the term that hasn't been detected.
    2. The `Category` box is intended for the user to choose the category of the entity highlighted by the user.
    3. The `Highlight` button is intended for the user to submit the higlighted term. The display will be automatically updated when the highlight is submitted. 
    
* The `Correct code map` is intended for the user to rectify mappings of sentence terms to RxNorm and ICD10 codes for drugs and conditions respectively.

    1. The `Drug (Condition)` box is inteded for users to select the detected Drug (Condition) for which they want to change the mapping.
    2. The `Show drug (condition) info` will display iformation introduced in the Drug (Condition) including mapping recommendations and the RxNorm (ICD10) code it is currently mapping to.
    3. The `Map to` box is intended for the user to write the RxNorm (ICD10) code they want the drug (condition) to map to.
    4. The `Update drug (condition)` button is intended for the user to submit the mapping update.
    

----

**Example question**

Questions where arguments will be detected:

    * Number of patients taking Aspirin
    * Number of patients having Insomnia
    
Questions where arguments will not be detected:
    
    * Number of patients by gender
    * Number of patients by race

----

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import config
from engine.pipeline import nlq2SqlTool
from ui.ui import UI

tool = nlq2SqlTool(config)
ui = UI(tool)
ui.main()

-----